# 支持向量机原理(四)SMO算法原理

在SVM的前三篇里，我们优化的目标函数最终都是一个关于α向量的函数。而怎么极小化这个函数，求出对应的α向量，进而求出分离超平面我们没有讲。本篇就对优化这个关于α向量的函数的SMO算法做一个总结。



## 1. 回顾SVM优化目标函数

我们首先回顾下我们的优化目标函数：


$$\underbrace{min}_{α} \frac{1}{2}\displaystyle\sum_{i=1,j=1}^{m}α_iα_jy_iy_jK(x_i∙x_j)-\displaystyle\sum_{i=1}^{m}α_i$$

$$s.t. \; \displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$0≤α_i≤C$$

我们的解要满足的KKT条件的对偶互补条件为：

$$α^∗_i(y_i(w^Tx_i+b)−1+ξ^∗_i)=0$$

根据这个KKT条件的对偶互补条件，我们有：

$$α^∗_i=0⇒y_i(w^∗∙ϕ(x_i)+b)≥1$$

$$0<α^∗_i<C⇒y_i(w^∗∙ϕ(x_i)+b)=1$$

$$α^∗_i=C⇒y_i(w^∗∙ϕ(x_i)+b)≤1$$

由于$$w^∗=\displaystyle\sum_{j=1}^{m}α^∗_jy_jϕ(x_j)$$,我们令

$$g(x)=w^∗∙ϕ(x)+b=\displaystyle\sum_{j=1}^{m}α^∗_jy_jK(x,x_j)+b^∗$$，

则有：

$$α^∗_i=0⇒y_ig(x_i)≥1$$

$$0<α^∗_i<C⇒y_ig(x_i)=1$$

$$α^∗_i=C⇒y_ig(x_i)≤1$$


## 2. SMO算法的基本思想

上面这个优化式子比较复杂，里面有m个变量组成的向量α需要在目标函数极小化的时候求出。直接优化时很难的。SMO算法则采用了一种启发式的方法。它每次只优化两个变量，将其他的变量都视为常数。由于$$\displaystyle\sum_{i=1}^{m}α_iy_i=0$$.假如将$$α_3,α_4,...,α_m$$　固定，

那么$$α_1,α_2$$之间的关系也确定了。这样SMO算法将一个复杂的优化算法转化为一个比较简单的两变量优化问题。

为了后面表示方便，我们定义$$K_{ij}=ϕ(x_i)∙ϕ(x_j)$$

由于$$α_3,α_4,...,α_m$$都成了常量，所有的常量我们都从目标函数去除，这样我们上一节的目标优化函数变成下式：

$$\underbrace{min}_{α_1,α_1}\frac{1}{2} K_{11}α_1^2+\frac{1}{2}K_{22}α_2^2+y_1y_2K_{12}α_1α_2−(α_1+α_2)+y_1α_1\displaystyle\sum_{i=3}^{m}y_iα_iK_{i1}+y_2α_2\displaystyle\sum_{i=3}^{m}y_iα_iK_{i2} $$

$$s.t. \; α_1y_1+α_2y_2=−\displaystyle\sum_{i=3}^{m}y_iα_i=ς$$

$$0≤α_i≤C \; i=1,2$$

## 3. SMO算法目标函数的优化

为了求解上面含有这两个变量的目标优化问题，我们首先分析约束条件，所有的$$α_1,α_2$$都要满足约束条件，然后在约束条件下求最小。

根据上面的约束条件$$α_1y_1+α_2y_2=ς \; 0≤α_i≤C \; i=1,2$$，又由于$$y_1,y_2$$均只能取值1或者-1, 这样α1,α2在[0,C]和[0,C]形成的盒子里面，并且两者的关系直线的斜率只能为1或者-1，也就是说$$α_1,α_2$$的关系直线平行于[0,C]和[0,C]形成的盒子的对角线，如下图所示：

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537856019172_6h16zO4pqo.jpg)

由于$$α_1,α_2$$的关系被限制在盒子里的一条线段上，所以两变量的优化问题实际上仅仅是一个变量的优化问题。不妨我们假设最终是$$α_2$$的优化问题。

由于我们采用的是启发式的迭代法，假设我们上一轮迭代得到的解是$$α^{old}_1,α^{old}_2$$，

假设沿着约束方向$$\alpha_2$$未经剪辑的解是$$\alpha^{new,unc}_2$$.

本轮迭代完成后的解为$$\alpha^{new}_1, \alpha^{new}_2$$

由于$$\alpha^{new}_2$$必须满足上图中的线段约束。

假设L和H分别是上图中$$\alpha^{new}_2$$所在的线段的边界。那么很显然我们有：

$$L≤α^{new}_2≤H$$

而对于L和H，我们也有限制条件如果是上面左图中的情况，则

$$L=max(0,α^{old}_2−α^{old}_1) \;\;\; H=min(C,C+α^{old}_2−α^{old}_1)$$

如果是上面右图中的情况，我们有：

$$L=max(0,α^{old}_2+α^{old}_1−C)H=min(C,α^{old}_2+α^{old}_1)$$

也就是说，假如我们通过求导得到的$$\alpha^{new,unc}_2$$，

则最终的$$\alpha^{new}_2$$应该为：

$$
\alpha^{new,unc}_2 = \begin{cases}
   H & \alpha^{new,unc}_2 > H \\
   \alpha^{new,unc}_2 &  L \le \alpha^{new,unc}_2 \le H \\
   L & \alpha^{new,unc}_2 \le L
\end{cases}
$$


那么如何求出$$\alpha^{new,unc}_2$$呢？

很简单，我们只需要将目标函数对$$\alpha_2$$求偏导数即可。

首先我们整理下我们的目标函数。

为了简化叙述，我们令

$$E_i=g(x_i)−y_i=\displaystyle\sum_{j=1}^{m}α^∗_jy_jK(x_i,x_j)+b−y_i$$

其中$$g(x)$$就是我们在第一节里面的提到的

$$g(x)=w^∗∙ϕ(x)+b=\displaystyle\sum_{j=1}^{m}α^∗_jy_jK(x,x_j)+b^∗$$

我们令

$$v_i=\displaystyle\sum_{j=3}^{m}y_jα_jK(x_i,xj)=g(x_i)−\displaystyle\sum_{j=1}^{2}y_jα_jK(x_i,x_j)−b$$


这样我们的优化目标函数进一步简化为：

$$W(α_1,α_2)=\frac{1}{2}K_{11}α^2_1+\frac{1}{2}K_{22}α_2^2+y_1y_2K_{12}α_1α_2−(α_1+α_2)+y_1α_1v_1+y_2α_2v_2$$

由于$$α_1y_1+α_2y_2=ς$$，并且$$y^2_i=1$$，

可以得到$$\alpha_1$$用$$\alpha_2$$表达的式子为：

$$α_1=y1(ς−α_2y_2)$$

将上式带入我们的目标优化函数，就可以消除$$\alpha_1$$,得到仅仅包含$$\alpha_2$$的式子。

$$W(α_2)=\frac{1}{2}K_{11}(ς−α_2y_2)^2+\frac{1}{2}K_{22}α_2^2+y_2K_{12}(ς−α_2y_2)α_2−(α_1+α_2)+(ς−α_2y_2)v_1+y_2α_2v_2$$

忙了半天，我们终于可以开始求$$α^{new,unc}_2$$了，

现在我们开始通过求偏导数来得到$$α^{new,unc}_2$$。

$$\frac{∂W}{∂α_2}=K_{11}α_2+K_{22}α_2−2K_{12}α_2−K_{11}ςy_2+K_{12}ςy_2+y_1y_2−1−v_1y_2+y_2v_2=0$$

整理上式有：

$$(K_{11}+K_{22}−2K_{12})α_2=y_2(y_2−y_1+ςK_{11}−ςK_{12}+v_1−v_2)$$

$$=y_2(y_2−y_1+ςK_{11}−ςK_{12}+(g(x_1)−\displaystyle\sum_{j=1}^{2}y_jα_jK_{1j}−b)−(g(x_2)−\displaystyle\sum_{j=1}^{2}y_jα_jK_{2j}−b))$$


将$$ς=α_1y_1+α_2y_2$$带入上式，我们有：

$$
\begin{aligned}
(K_{11}+K_{22}−2K_{12})α^{new,unc}_2 &= y_2((K_{11}+K_{22}−2K_{12})α^{old}_2y_2+y_2−y_1+g(x_1)−g(x_2)) \\ 
 &= (K_{11}+K_{22}−2K_{12})α^{old}_2+y_2(E_1−E_2)
\end{aligned}
$$



我们终于得到了$$α^{new,unc}_2$$的表达式：

$$α_2^{new,unc}=α_2^{old}+y_2(E_1−E_2)K_{11}+K_{22}−2K_{12})$$

利用上面讲到的$$\alpha_2^{new,unc}$$和$$\alpha_2^{new}$$的关系式，我们就可以得到我们新的$$\alpha_2^{new}$$了。利用$$alpha_2^{new}$$和$$\alpha_1^{new}$$的线性关系，我们也可以得到新的$$\alpha_1^{new}$$。





## 4. SMO算法两个变量的选择

SMO算法需要选择合适的两个变量做迭代，其余的变量做常量来进行优化，那么怎么选择这两个变量呢？

### 4.1 第一个变量的选择

SMO算法称选择第一个变量为外层循环，这个变量需要选择在训练集中违反KKT条件最严重的样本点。对于每个样本点，要满足的KKT条件我们在第一节已经讲到了： 

$$α^∗_i=0⇒y_ig(x-i)≥1$$

$$0<α^∗_i<C⇒y_ig(x_i)=1$$

$$α^∗_i=C⇒y_ig(x_i)≤1$$

一般来说，我们首先选择违反$$0<α^∗_i<C⇒y_ig(x_i)=1$$这个条件的点。如果这些支持向量都满足KKT条件，再选择违反$$α^∗_i=0⇒y_ig(x_i)≥1$$和$$α^∗_i=C⇒y_ig(x_i)≤1$$的点。

### 4.2 第二个变量的选择

SMO算法称选择第二一个变量为内层循环，假设我们在外层循环已经找到了$$\alpha_1$$, 第二个变量$$\alpha_2$$的选择标准是让$$|E1−E2|$$有足够大的变化。由于$$α_1$$定了的时候,$$E_1$$也确定了，所以要想$$|E1−E2|$$最大，只需要在$$E_1$$为正时，选择最小的$$E_i$$作为$$E_2$$， 在$$E_1$$为负时，选择最大的$$E_i$$作为$$E_2$$，可以将所有的$$E_i$$保存下来加快迭代。

如果内存循环找到的点不能让目标函数有足够的下降， 可以采用遍历支持向量点来做$$\alpha_2$$,直到目标函数有足够的下降， 如果所有的支持向量做$$\alpha_2$$都不能让目标函数有足够的下降，可以跳出循环，重新选择$$α_1$$

### 4.3 计算阈值b和差值$$E_i$$　

在每次完成两个变量的优化之后，需要重新计算阈值b。当$$0<α^{new}_1<C$$时，我们有

$$y_1−\displaystyle\sum_{i=1}^{m}α_iy_iK_{i1}−b_1=0$$

于是新的$$b^{new}_1$$为：

$$b^{new}_1=y_1−\displaystyle\sum_{i=3}^{m}α_iy_iK_{i1}−α^{new}_1y_1K_{11}−α^{new}_2y_2K_{21}$$

计算出$$E_1$$为：

$$E_1=g(x_1)−y_1=\displaystyle\sum_{i=3}^{m}α_iy_iK_{i1}+α^{old}_1y_1K_{11}+α^{old}_2y_2K_{21}+b^{old}−y_1$$

可以看到上两式都有$$y_1−\displaystyle\sum_{i=3}^{m}α_iy_iK_{i1}$$，因此可以将$$b_1^{new}$$用$$E_1$$表示为：

$$b^{new}_1=−E_1−y_1K_{11}(α^{new}_1−α^{old}_1)−y_2K_{21}(α^{new}_2−α^{old}_2)+b^{old}$$

同样的，如果$$0<α^{new}_2<C$$, 那么有：

$$b_2^{new}=−E_2−y_1K_{12}(α_1^{new}−α_1^{old})−y_2K_{22}(α_2^{new}−α_2^{old})+b^{old}$$

最终的$$b^{new}$$为：

$$b^{new}=\frac{b_1^{new}+b_2^{new}}{2}$$

得到了bnew我们需要更新$$E_i$$:

$$E_i=\displaystyle\sum_Sy_jα_jK(x_i,x_j)+b^{new}−y_i$$

其中，S是所有支持向量$$x_j$$的集合。

好了，SMO算法基本讲完了，我们来归纳下SMO算法。





## 5. SMO算法总结

输入是m个样本$$(x_1,y_1),(x_2,y_2),...,(x_m,y_m)$$,,其中x为n维特征向量。$$y$$为二元输出，值为1，或者-1.精度e。

输出是近似解$$α$$

1)取初值$$α_0=0,k=0$$

2)按照4.1节的方法选择$$\alpha_1^k$$,接着按照4.2节的方法选择$$\alpha_2^k$$，求出新的$$α_2^{new,unc}$$。

$$\alpha_2^{new,unc}=α_2^k+\frac {y_2(E_1−E_2)}{K_{11}+K_{22}−2K_{12}}$$

3)按照下式求出$$α_2^{k+1}$$

$$
\alpha_2^{k+1} = \begin{cases}
   H & L \le \alpha_2^{new,unc} \gt H \\
   \alpha_2^{new,unc} &  L \le \alpha_2^{new,unc} \le H \\
   L & \alpha_2^{new,unc} \lt L
\end{cases}
$$

4)利用$$\alpha_2^{k+1}$$和$$\alpha_1^{k+1}$$的关系求出$$α_1^{k+1}$$

5)按照4.3节的方法计算$$b^{k+1}$$和$$E_i$$

6）在精度e范围内检查是否满足如下的终止条件：

$$\displaystyle\sum_{i=1}^{m}\alpha_iy_i=0$$

$$0≤αi≤C,i=1,2...m$$

$$α_i^{k+1}=0⇒y_ig(x_i)≥1$$

$$0<α_i^{k+1}<C⇒y_ig(x_i)=1$$

$$α_i^{k+1}=C⇒y_ig(x_i)≤1$$

7)如果满足则结束，返回$$α^{k+1}$$,否则转到步骤2）

SMO算法终于写完了，这块在以前学的时候是非常痛苦的，不过弄明白就豁然开朗了。希望大家也是一样。写完这一篇， SVM系列就只剩下支持向量回归了，胜利在望！


文章转载自：[刘建平Pinard 支持向量机原理(四)SMO算法原理](https://www.cnblogs.com/pinard/p/6111471.html)

